## **Modeling Car Insurances**
Insurance companies invest a lot of time and money into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

(`Source: https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf`) 

Knowing all of this, a car insurance have requested my services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked me to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied me with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.

## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

### Tasks
- Identify the single feature of the data that is the best predictor of whether a customer will put in a claim (the "outcome" column), excluding the "id" column.

- Store as a DataFrame called best_feature_df, containing columns named "best_feature" and "best_accuracy" with the name of the feature with the highest accuracy, and the respective accuracy score.

In [ ]:
# Importing necessary libraries
import pandas as pd
from statsmodels.formula.api import logit


In [ ]:
# Reading csv file

car_df = pd.read_csv('car_insurance.csv')

print(car_df.head(3))

In [ ]:
# Cleaning and transforming the dataset

#print(car_df.isna().sum())
# Imputing median value to NaN values at "credit_score" & "annual_milage"
car_df['credit_score'] = car_df['credit_score'].fillna(car_df['credit_score'].median())
car_df['annual_mileage'] = car_df['annual_mileage'].fillna(car_df['annual_mileage'].median())
# Ensure none NaN value exists in the columns that were found
assert car_df['credit_score'].isna().sum() == 0
assert car_df['annual_mileage'].isna().sum() == 0

# Difining the maps to map each categorical column with

vehicle_type_map = {
    'sedan' : '0', 
    'sports car' : '1' 
}

vehicle_year_map = {
    'before 2015' : '0',
    'after 2015' : '1'
}

driving_exp_map = {
    '0-9y' : '0',
    '10-19y' : '1',
    '20-29y' : '2',
    '30y+' : '3' 
}

education_map = {
    'none': '0',
    'high school' : '1',
    'university' : '2'
}

income_map = {
    'poverty' : '0',
    'working class' : '1',
    'middle class' : '2',
    'upper class' : '3' 
}

# Transform the categorical columns
car_df['driving_experience'] = car_df['driving_experience'].map(driving_exp_map)
car_df['income'] = car_df['income'].map(income_map)
car_df['education'] = car_df['education'].map(education_map)
car_df['vehicle_type'] = car_df['vehicle_type'].map(vehicle_type_map)
car_df['vehicle_year'] = car_df['vehicle_year'].map(vehicle_year_map)

#Before starting the models, "id", "postal_code" should be dropped
modeldata_cars = car_df[['age', 'gender', 'driving_experience', 'education', 
                         'income', 'credit_score', 
                         'vehicle_type', 'vehicle_ownership', 
                         'vehicle_year', 'married', 'children', 
                         'annual_mileage', 'speeding_violations', 'duis',
                         'past_accidents', 'outcome']]

print(modeldata_cars.head(10))


In [ ]:
# Defining 2 very helpful functions

def run_logit(expl_vars, response_var, data):
    """
    Performs logistic regression for each of 
    the explanatory variables seperately and returns 
    the confusion matrices for every model.

    Args:
        expl_vars (list): A list with the names of the explanatory variables
        response_var (str): The name of the response variable
        data (pd.DataFrame): DataFrame containing the data

    Returns:
        x (dict): The dictionary with the confusion matrices for every logistic regression performed
    """
    x = {}
    for i in expl_vars:
        # Creating the formula for every explanatory variable
        formula = f"{response_var} ~ {i}"
        
        # Creating and fitting the model
        model = logit(formula=formula, data=data)
        fit = model.fit(disp=0)    
        result = fit.pred_table()
        # Storing the results in the dictionary 
        x[i] = result
    return x


def acc_find(x):
    """
    Calculates the accuracy of the pred_tables given from each explanatory variable and returns it 

    Args:
        x (dict): A dictionary containing all the explanatory variables as keys and the pred_tables of logistic regressions as values.
    
    Returns:
        acc_results (dict): The dictionary containing the explanatory variables as keys and the calculated accuracies for each model fitted before as values.
    """
    acc_results = {}
    for key, value in x.items():
        accuracy = (value[0,0]+value[1,1])/(value[0,0]+value[0,1]+value[1,0]+value[1,1])
        acc_results[key] = accuracy
    
    return acc_results
        


In [ ]:
# Get the features and the target data

response_var = 'outcome'

expl_vars = ['age', 'gender', 'driving_experience', 'education', 
                         'income', 'credit_score', 
                         'vehicle_type', 'vehicle_ownership', 
                         'vehicle_year', 'married', 'children', 
                         'annual_mileage', 'speeding_violations', 'duis',
                         'past_accidents']

# Apply the run_logit function
logit_tables_dict = run_logit(expl_vars=expl_vars, response_var=response_var, data=modeldata_cars)

# Apply the acc_find function
accuracy_results = acc_find(logit_tables_dict)




In [ ]:
# Get the max key-value pair from the results
# This is the best predictor of the features
max_acc = max(accuracy_results.values())
best_predictor = max(accuracy_results, key=accuracy_results.get)

In [ ]:
# Store as a DataFrame
best_feature_df = pd.DataFrame({'best_feature' : [best_predictor],
                                'best_accuracy' : [max_acc]})

print(best_feature_df)